In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import permutations,combinations
from collections import Counter
import requests
import joblib

In [2]:
# all_data = []
# def get_paginated_results(url,all_data):
#     response = requests.get(url)
#     res = response.json()
#     if response.status_code == 200:
#         if res['data']["results"]:
#             all_data += res['data']["results"]
#     if res['data'] == None or res['data']['next'] == None:
#         return all_data
    
#     return get_paginated_results(res['data']['next'],all_data)

In [3]:
# url='https://api-########-uat.########.com/dynamic/purchase-history-list'
# all_data_res = get_paginated_results(url,all_data)


In [4]:
# purchaseData = pd.DataFrame(all_data_res)
# purchaseData

In [5]:
purchaseData=joblib.load("purchasedata.pkl")

In [6]:
purchaseData

,id,remote_id,order_id,cart_id,customer_code,variant_id,variant_code,variant_name,total_quantity,branch_id,...,segment,segment_name,brand_name,brand_code,offer_id,offer_value,uom_name,uom_code,specialgroup_name,inventory_update
0,9,365,167,NaN,RGC1658833657,43452,None,None,1,BSNU1000,...,None,[],None,None,None,None,None,None,[],False
1,144,10098,10098,10098.0,RGC1658833657,43452,None,None,1,BSNU1000,...,None,[],None,None,None,None,None,None,[],False
2,1,14,37,NaN,RGC1658833657,4492,None,None,1,BSNU1000,...,None,[],None,None,None,None,None,None,[],False
3,2,15,38,NaN,RGC1658833657,4492,None,None,1,BSNU1000,...,None,[],None,None,None,None,None,None,[],False
4,3,26,49,NaN,RGC1658833657,4492,None,None,1,BSNU1000,...,None,[],None,None,None,None,None,None,[],False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,271,465,281,NaN,RGC1658833657,4492,None,None,1,BSNU1000,...,None,[],None,None,None,None,None,None,[],False
271,272,466,282,NaN,RGC1658833657,31966,None,None,1,BSNU1006,...,None,[],None,None,None,None,None,None,[],False
272,273,467,283,NaN,RGC1658833657,31927,None,None,1,BSNU1006,...,None,[],None,None,None,None,None,None,[],False
273,274,468,283,NaN,RGC1658833657,31966,None,None,1,BSNU1006,...,None,[],None,None,None,None,None,None,[],False


In [7]:
dataset=purchaseData[["order_id","variant_id"]]

In [8]:
def find_pairs(x):
    pairs = pd.DataFrame(list(permutations(x.values,2)),columns=["A","B"])
    return pairs

In [9]:
dataset_combo =dataset.groupby('order_id')['variant_id'].apply(find_pairs).reset_index(drop=True)
dataset_combo

,A,B
0,16752,31966
1,16752,31927
2,31966,16752
3,31966,31927
4,31927,16752
5,31927,31966
6,31927,16752
7,31927,31966
8,16752,31927
9,16752,31966


In [10]:
dataset_combo2 =dataset_combo.groupby(['A','B']).size()
dataset_combo2

A      B    
16752  23867     1
       31927     7
       31966     7
23867  16752     1
       31927     1
       31966     1
31927  16752     7
       23867     1
       31966    10
31966  16752     7
       23867     1
       31927    10
dtype: int64

In [11]:
datasetup =dataset_combo2.reset_index()
datasetup.columns = ['A','B',"Size"]
datasetup.sort_values(by='Size',ascending =False, inplace =True)
datasetup

,A,B,Size
8,31927,31966,10
11,31966,31927,10
1,16752,31927,7
2,16752,31966,7
6,31927,16752,7
9,31966,16752,7
0,16752,23867,1
3,23867,16752,1
4,23867,31927,1
5,23867,31966,1


In [12]:
datasetup[datasetup["A"]=="31927"]

,A,B,Size
8,31927,31966,10
6,31927,16752,7
7,31927,23867,1


In [16]:
def frequentlyPurchased(product):
    pro=datasetup[datasetup["A"]==product]
    pro.sort_values(by='Size',ascending =False, inplace =True)
    b=[]
    b.append(product)
    return b+list(pro["B"])[0:5]
    

In [21]:
frequentlyPurchased("31927")

['31927', '31966', '16752', '23867']